In [115]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Global Economic Forecast Hackathon with Texas A&M Aggie Data Science")

Dataset "Statistical Performance Indicators" loaded to the kernel as statistical_performance_indicators
Key Name                       Value Type     
---------------------------------------------
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: e7f12f65-288e-4ff6-9b27-598ce44275c3, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [117]:
%%ag
x_train = statistical_performance_indicators["train_x"]
y_train = statistical_performance_indicators["train_y"]
x_test = statistical_performance_indicators["test_x"]

In [118]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from op_tensorflow import PrivateKerasModel, PrivateDataLoader

# Efficient Sequential model with privacy constraints in mind
seqM = Sequential([
    # Input block
    Dense(256, input_shape=(24,)),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.1),

    # First dense block
    Dense(256),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.1),

    # Second dense block
    Dense(128),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.1),

    # Final refinement layer
    Dense(64),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.1),

    # Output layer
    Dense(1, activation='sigmoid')
])

# Create DP keras model with strict privacy parameters
dp_model = PrivateKerasModel(
    model=seqM,
    l2_norm_clip=0.8,  # Reduced for privacy budget
    noise_multiplier=0.7  # Increased for stronger privacy guarantees
)

# Optimizer with conservative learning rate
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0005,  # Reduced learning rate
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
)

# Compile model
dp_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.10/site-packages/tensorflow/__init__.py:30: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils

2024-11-17 13:31:24.219661: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 13:31:24.222872: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 13:31:24.335447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 13:31:24.335484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unabl

In [119]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train , label_df=y_train, batch_size=64)

In [120]:
%%ag
dp_model.fit(x=data_loader, epochs=30, target_delta=0.5)

Epoch 1/30

58/58 - 11s - loss: 0.4897 - accuracy: 0.7578 - 11s/epoch - 183ms/step

Epoch 2/30

58/58 - 3s - loss: 0.4217 - accuracy: 0.7923 - 3s/epoch - 48ms/step

Epoch 3/30

58/58 - 2s - loss: 0.3928 - accuracy: 0.8107 - 2s/epoch - 43ms/step

Epoch 4/30

58/58 - 2s - loss: 0.3571 - accuracy: 0.8399 - 2s/epoch - 41ms/step

Epoch 5/30

58/58 - 2s - loss: 0.3494 - accuracy: 0.8392 - 2s/epoch - 43ms/step

Epoch 6/30

58/58 - 2s - loss: 0.3321 - accuracy: 0.8475 - 2s/epoch - 41ms/step

Epoch 7/30

58/58 - 2s - loss: 0.3143 - accuracy: 0.8515 - 2s/epoch - 41ms/step

Epoch 8/30

58/58 - 2s - loss: 0.3045 - accuracy: 0.8612 - 2s/epoch - 41ms/step

Epoch 9/30

58/58 - 3s - loss: 0.2959 - accuracy: 0.8677 - 3s/epoch - 43ms/step

Epoch 10/30

58/58 - 3s - loss: 0.2969 - accuracy: 0.8687 - 3s/epoch - 45ms/step

Epoch 11/30

58/58 - 2s - loss: 0.2658 - accuracy: 0.8821 - 2s/epoch - 43ms/step

Epoch 12/30

58/58 - 2s - loss: 0.2687 - accuracy: 0.8778 - 2s/epoch - 41ms/step

Epoch 13/30

58/58 - 2

In [121]:
%%ag
test=PrivateDataFrame(x_test).copy()
y_pred = dp_model.predict(test, label_columns=["output"])

13/13 [==============================] - 1s 15ms/step



In [122]:
%%ag
# Note that the predictions are a float scalar, so we scale it!
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

In [124]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.968402874717081, 'logs': {'BIN_ACC': 0.968402874717081, 'LIN_EPS': -0.0}}

